# Tensorflow Mechanics 101

[Tensorflow Mechanics 101](https://www.tensorflow.org/versions/r0.10/tutorials/mnist/tf/index.html)

## Tutorial Files

| File                | Purpose |
|:--------------------|:--------|
| [mnist.py](https://github.com/tensorflow/tensorflow/blob/r0.10/tensorflow/examples/tutorials/mnist/mnist.py) | The code to build a fully-connected MNIST model. |
| [fully_connected_feed.py](https://github.com/tensorflow/tensorflow/blob/r0.10/tensorflow/examples/tutorials/mnist/fully_connected_feed.py) | The main code to train the built MNIST model against the downloaded dataset using a feed dictionary. |

In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import os.path
import time
import math

from six.moves import xrange
import tensorflow as tf

from tensorflow.examples.tutorials.mnist import input_data
from tensorflow.examples.tutorials.mnist import mnist

In [2]:
flags = tf.app.flags
FLAGS = flags.FLAGS
flags.DEFINE_float('learning_rate', 0.01, 'Initial learning rate.')
flags.DEFINE_integer('max_steps', 2000, 'Number of steps to run trainer.')
flags.DEFINE_integer('hidden1', 128, 'Number of units in hidden layer 1.')
flags.DEFINE_integer('hidden2', 32, 'Number of units in hidden layer 2.')
flags.DEFINE_integer('batch_size', 100, 'Batch size. Must divide evenly into the dataset sizes.')
flags.DEFINE_string('train_dir', 'data', 'Directory to put the training data.')
flags.DEFINE_boolean('fake_data', False, 'If true, uses fake data for unit testing.')

## Download

In [3]:
data_sets = input_data.read_data_sets(FLAGS.train_dir, FLAGS.fake_data)

Extracting data/train-images-idx3-ubyte.gz
Extracting data/train-labels-idx1-ubyte.gz
Extracting data/t10k-images-idx3-ubyte.gz
Extracting data/t10k-labels-idx1-ubyte.gz


## Inputs and Placeholders

In [4]:
images_placeholder = tf.placeholder(tf.float32, shape=(FLAGS.batch_size, mnist.IMAGE_PIXELS))
labels_placeholder = tf.placeholder(tf.int32, shape=(FLAGS.batch_size))

## Build the Graph

In [5]:
tf.Graph().as_default()

In [12]:
NUM_CLASSES = 10
IMAGE_SIZE = 28
IMAGE_PIXELS = IMAGE_SIZE * IMAGE_SIZE

In [9]:
with tf.name_scope('hidden1'):
    weights = tf.Variable(tf.truncated_normal([IMAGE_PIXELS, FLAGS.hidden1], stddev=1.0 / math.sqrt(float(IMAGE_PIXELS))), name='weights')
    biases = tf.Variable(tf.zeros([FLAGS.hidden1]), name='biases')
    hidden1 = tf.nn.relu(tf.matmul(images_placeholder, weights) + biases)

In [11]:
with tf.name_scope('hidden2'):
    weights = tf.Variable(tf.truncated_normal([FLAGS.hidden1, FLAGS.hidden2], stddev=1.0 / math.sqrt(float(FLAGS.hidden1))), name='weights')
    biases = tf.Variable(tf.zeros([FLAGS.hidden2]), name='biases')
    hidden2 = tf.nn.relu(tf.matmul(hidden1, weights) + biases)

In [13]:
with tf.name_scope('softmax_liner'):
    weights = tf.Variable(tf.truncated_normal([FLAGS.hidden2, NUM_CLASSES], stddev=1.0 / math.sqrt(float(FLAGS.hidden2))), name='weights')
    biases = tf.Variable(tf.zeros([NUM_CLASSES], name='biases'))
    logits = tf.matmul(hidden2, weights) + biases

## Loss

In [18]:
labels = tf.to_int64(labels_placeholder)
cross_entropy = tf.nn.sparse_softmax_cross_entropy_with_logits(logits, labels, name='xentropy')

In [19]:
loss = tf.reduce_mean(cross_entropy, name='xentropy_mean')

## Training

In [20]:
tf.scalar_summary(loss.op.name, loss)

<tf.Tensor 'ScalarSummary_2:0' shape=() dtype=string>

In [21]:
optimizer = tf.train.GradientDescentOptimizer(FLAGS.learning_rate)

In [22]:
global_step = tf.Variable(0, name='global_step', trainable=False)
train_op = optimizer.minimize(loss, global_step=global_step)

## Train the Model